# term1 アンサンブル学習

## 1.このSprintについて
### Sprintの目的
アンサンブル学習について理解する

### どのように学ぶか
スクラッチでアンサンブル学習の各種手法を実装していきます
## 2.アンサンブル学習
3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。


- ブレンディング
- バギング
- スタッキング

### 小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。


train.csvを学習用（train）8割、検証用（val）2割に分割してください。

### scikit-learn
単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。

sklearn.linear_model.LinearRegression — scikit-learn 0.21.3 documentation


sklearn.svm.SVR — scikit-learn 0.21.3 documentation


sklearn.tree.DecisionTreeRegressor — scikit-learn 0.21.3 documentation

## 3.ブレンディング

### 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

### ブレンディングとは
ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。


- 手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
- ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
- 入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。


回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。


《補足》


分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

In [30]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format
import matplotlib.pyplot as plt

In [50]:
df = pd.read_csv("train.csv")
index_array = df.columns
index_list = index_array.tolist()
index_list.remove("GrLivArea")
index_list.remove("YearBuilt")
index_list.remove("SalePrice")
df.drop(columns=index_list, inplace=True)
#df.columns
X = df.iloc[:, :2].values
y = df.iloc[:, 2:].values

In [51]:
# yには対数変換をかけておく
y = np.log(y)

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [53]:
# 標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_val_std = scaler.transform(X_val)

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [54]:
# 線形回帰 学習と推定
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression().fit(X_train_std, y_train)
yp_linear = linear_model.predict(X_val_std)
yp_linear.shape

(292, 1)

In [55]:
# SVM 学習と推定
from sklearn.svm import SVR
svr_model = SVR()
svr_model.fit(X_train_std, y_train)
yp_svr = svr_model.predict(X_val_std)
yp_svr.shape

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(292,)

In [56]:
# 決定木 学習と推定
from sklearn.tree import DecisionTreeRegressor
d_tree_model = DecisionTreeRegressor()
d_tree_model.fit(X_train_std, y_train)
yp_d_tree = d_tree_model.predict(X_val_std)
yp_d_tree.shape

(292,)

In [57]:
# 平均二乗誤差で評価
from sklearn.metrics import mean_squared_error
error_linear = mean_squared_error(y_val, yp_linear)
error_svr = mean_squared_error(y_val, yp_svr)
error_d_tree = mean_squared_error(y_val, yp_d_tree)

In [58]:
# 表作成用
error_df = pd.DataFrame({"LinearRegression":[error_linear], "SVM" : [error_svr], "決定木" : [error_d_tree]},
                        index=["平均二乗誤差"])

In [59]:
error_df

,LinearRegression,SVM,決定木
平均二乗誤差,0.036798,0.035652,0.058852


In [60]:
 yp_d_tree.shape

(292,)

In [67]:
# 線型回帰とSVM、決定木を足して平均を取る
LSDt_yp = np.concatenate([yp_linear.reshape(len(yp_linear), 1), yp_svr.reshape(len(yp_svr), 1 ),
                          yp_d_tree.reshape(len(yp_d_tree),1)], 1)

LSDt_mean = np.mean(LSDt_yp, 1)
LSDt_mean.shape

(292,)

In [68]:
# 平均二乗誤差で評価
error_LSDt_m = mean_squared_error(y_val, LSDt_mean)

In [70]:
error_df = error_df.assign(線型SVM決定木=[error_LSDt_m])
error_df

,LinearRegression,SVM,決定木,線型SVM決定木
平均二乗誤差,0.036798,0.035652,0.058852,0.033961


In [82]:
w = np.array([0.3, 0.5, 0.2])
LSDt_w_yp = np.average(LSDt_yp, axis=1 ,weights=w)
LSDt_w_yp.shape

(292,)

In [83]:
# 平均二乗誤差で評価
error_LSDt_wm = mean_squared_error(y_val, LSDt_w_yp)

In [84]:
error_df = error_df.assign(線S木w=[error_LSDt_wm])
error_df

,LinearRegression,SVM,決定木,線型SVM決定木,線S木w
平均二乗誤差,0.036798,0.035652,0.058852,0.033961,0.033209


In [85]:
# 線型回帰とSVMを足して平均を取る
LS_yp = np.concatenate([yp_linear.reshape(len(yp_linear), 1), yp_svr.reshape(len(yp_svr), 1 )], 1)
LS_mean = np.mean(LS_yp, 1)
LS_mean.shape

(292,)

In [86]:
# 平均二乗誤差で評価
error_LS_m = mean_squared_error(y_val, LS_mean)

In [87]:
error_df = error_df.assign(線S=[error_LS_m])
error_df

,LinearRegression,SVM,決定木,線型SVM決定木,線S木w,線S
平均二乗誤差,0.036798,0.035652,0.058852,0.033961,0.033209,0.034751


線型回帰、SVM、決定木の単体モデルの平均二乗誤差と、<br>
①3つのモデル（線型回帰、SVM、決定木）の平均<br>
②3つのモデル（線型回帰、SVM、決定木）の加重平均<br>
③２つのモデル（線型回帰、SVM）の平均で比較をした<br>

【考察】<br>
単体に比べ①〜③全てで平均二乗誤差が小さくなった。
成績の良かった２つのモデルの平均より、少し成績が悪くても３つのモデルの平均を取った方が平均二乗誤差が小さくなった。

## 4.バギング
### 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

### バギングとは
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。

sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation


scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。


推定結果の平均をとる部分はブースティングと同様の実装になります。

In [115]:
def bagging_func(n, X_train_std, y_train, X_val_std):
    for i in range(n):
        X_t, X_v, y_t, y_v = train_test_split(X_train_std, y_train, test_size=0.2, shuffle=True)
        # X_train_stdの一部を学習
        model = DecisionTreeRegressor()
        model.fit(X_t, y_t)
        # X_val_stdから予測
        yp_v = model.predict(X_val_std)
        if i == 0:
            yp_all = yp_v.reshape(len(yp_v),1)
        else:
            yp_all = np.concatenate([yp_all,yp_v.reshape(len(yp_v),1)], 1)
    #print("yp_all.shape", yp_all.shape)       
    yp_mean = np.mean(yp_all, 1)
    #print("yp_mean", yp_mean)
    return yp_mean        

In [120]:
bagging_yp_1 = bagging_func(1, X_train_std, y_train, X_val_std)
bagging_yp_5 = bagging_func(5, X_train_std, y_train, X_val_std)
bagging_yp_10 = bagging_func(10, X_train_std, y_train, X_val_std)
bagging_yp_20 = bagging_func(20, X_train_std, y_train, X_val_std)
bagging_yp_50 = bagging_func(50, X_train_std, y_train, X_val_std)
bagging_yp_100 = bagging_func(100, X_train_std, y_train, X_val_std)

In [121]:
error_bg_1 = mean_squared_error(y_val, bagging_yp_1)
error_bg_5 = mean_squared_error(y_val, bagging_yp_5)
error_bg_10 = mean_squared_error(y_val, bagging_yp_10)
error_bg_20 = mean_squared_error(y_val, bagging_yp_20)
error_bg_100 = mean_squared_error(y_val, bagging_yp_100)

In [122]:
# 表作成用
error_bg_df = pd.DataFrame({"1回" : [error_bg_1], "5回":[error_bg_5], "10回" : [error_bg_10],
                         "20回" : [error_bg_20], "50回" : [error_bg_50], "100回" : [error_bg_100]},
                        index=["平均二乗誤差"])

In [123]:
error_bg_df

,1回,5回,10回,20回,50回,100回
平均二乗誤差,0.059500,0.044653,0.042090,0.039258,0.039075,0.039590


【考察】<br>
５０回まではサンプルを増やすと良い結果が得られたが、100回では少し精度が下がった。

## 5.スタッキング
### 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

### スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは $K_0=3, M_0=2$ 程度にします。

《学習時》


（ステージ $0$ ）


学習データを $K_0$ 個に分割する。
分割した内の $(K_0 - 1)$ 個をまとめて学習用データ、残り $1$ 個を推定用データとする組み合わせが $K_0$ 個作れる。
あるモデルのインスタンスを $K_0$ 個用意し、異なる学習用データを使い学習する。
それぞれの学習済みモデルに対して、使っていない残り $1$ 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
さらに、異なるモデルのインスタンスも $K_0$ 個用意し、同様のことを行う。モデルが $M_0$ 個あれば、 $M_0$ 個のブレンドデータが得られる。

（ステージ $n$ ）


ステージ $n-1$ のブレンドデータを$M_{n-1}$ 次元の特徴量を持つ学習用データと考え、 $K_n$ 個に分割する。以下同様である。

（ステージ $N$ ）＊最後のステージ


ステージ $N-1$ の $M_{N-1}$ 個のブレンドデータを$M_{N-1}$ 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

《推定時》


（ステージ $0$ ）


テストデータを $K_0×M_0$ 個の学習済みモデルに入力し、$K_0×M_0$ 個の推定値を得る。これを $K_0$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $n$ ）


ステージ $n-1$ で得たブレンドテストを $K_n×M_n$ 個の学習済みモデルに入力し、$K_n×M_n$ 個の推定値を得る。これを $K_n$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $N$ ）＊最後のステージ


ステージ $N-1$ で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

In [199]:
K=3
# 学習フェーズ 1層目
from sklearn.model_selection import KFold
kf = KFold(n_splits=K, shuffle=True, random_state=71)
m_instance_list = []
va_index = []
count = 0
for tr_idx, va_idx in kf.split(X_train_std):
    tr_x, va_x = X_train_std[tr_idx], X_train_std[va_idx]
    tr_y, va_y = y_train[tr_idx], y_train[va_idx]
    # 線型回帰
    lin_model = LinearRegression()
    lin_model.fit(tr_x, tr_y)
    lin_pred = lin_model.predict(va_x)
    
    #決定木
    dt_model = DecisionTreeRegressor()
    dt_model.fit(tr_x, tr_y)
    dt_pred = dt_model.predict(va_x)
    
    #SVM
    sv_model = SVR()
    sv_model.fit(tr_x, tr_y)
    sv_pred = sv_model.predict(va_x)
    
    # インスタンスの保存
    m_instance_list.append(lin_model)
    m_instance_list.append(dt_model)
    m_instance_list.append(sv_model)
    
    # valのindexの保存
    va_index.append(va_idx)
    
    # ブレンドデータの作成
    if count == 0:
        lin_pred_all = lin_pred.reshape(len(lin_pred),1)
        dt_pred_all = dt_pred.reshape(len(dt_pred),1)
        sv_pred_all = sv_pred.reshape(len(sv_pred),1)
    else:
        lin_pred_all = np.concatenate([lin_pred_all,lin_pred.reshape(len(lin_pred),1)], 0)
        dt_pred_all = np.concatenate([dt_pred_all,dt_pred.reshape(len(dt_pred),1)], 0)
        sv_pred_all = np.concatenate([sv_pred_all,sv_pred.reshape(len(sv_pred),1)], 0)
    
    count += 1


/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [200]:
# indexのリストを1つのデータに
va_index = np.concatenate(va_index)
# 各モデルのブレンドデータも結合
m_preds = np.concatenate([lin_pred_all, dt_pred_all, sv_pred_all], 1)
# 順番に並べるためのindexを返す
order = np.argsort(va_index)
# ブレンドデータを元のサンプルの順番に並べ直す
m_pred_all = m_preds[order]

In [203]:
# 学習フェーズ 最終層
svm_model = SVR()
svm_model.fit(m_pred_all, y_train)

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [204]:
# 推論フェーズ
for i in range(K):
    # インスタンスを取り出す
    lin_m = m_instance_list[i]
    dt_m = m_instance_list[i + 1]
    sv_m = m_instance_list[i + 2]
    
    # 予測
    lin_p = lin_m.predict(X_val_std)
    dt_p = dt_m.predict(X_val_std)
    sv_p = sv_m.predict(X_val_std)
    
    # 各モデルのバリデーション分の予測データを作成
    if i == 0:
        lin_p_all = lin_p.reshape(len(lin_p),1)
        dt_p_all = dt_p.reshape(len(dt_p),1)
        sv_p_all = sv_p.reshape(len(sv_p),1)
    else:
        lin_p_all = np.concatenate([lin_p_all,lin_p.reshape(len(lin_p),1)], 1)
        dt_p_all = np.concatenate([dt_p_all,dt_p.reshape(len(dt_p),1)], 1)
        sv_p_all = np.concatenate([sv_p_all,sv_p.reshape(len(sv_p),1)], 1)

In [205]:
lin_p_m = np.mean(lin_p_all, 1)
dt_p_m = np.mean(dt_p_all, 1)
sv_p_m = np.mean(sv_p_all, 1)
#p_all = np.concatenate([lin_p_m.reshape(len(lin_p_m), 1), dt_p_m.reshape(len(dt_p_m), 1)], 1)
p_all = np.concatenate([lin_p_m.reshape(len(lin_p_m), 1), dt_p_m.reshape(len(dt_p_m), 1),
                       sv_p_m.reshape(len(sv_p_m), 1)], 1)
p_all.shape

(292, 3)

In [206]:
# 推論フェーズ ラストステージ
stacking_yp = svm_model.predict(p_all)

In [207]:
stacking_yp.shape

(292,)

In [208]:
# 平均二乗誤差で評価
error_stacking = mean_squared_error(y_val, stacking_yp)

In [209]:
error_stacking_df = pd.DataFrame({"LinearRegression":[error_linear], "SVM" : [error_svr],
                         "決定木" : [error_d_tree], "Stacking" : [error_stacking]},
                        index=["平均二乗誤差"])

In [210]:
error_stacking_df

,LinearRegression,SVM,決定木,Stacking
平均二乗誤差,0.036798,0.035652,0.058852,0.031665


【考察】<br>
スタッキング（1層目：線型回帰、SVM、決定木　2層目：SVM）を行った結果、単一モデルより精度が高くなった